In [ ]:
import glob
import pandas as pd
import numpy as np 
from astropy.time import Time
from astropy.table import Table
from astropy.io import fits
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import time
from scipy.stats import sem
from datetime import datetime, timezone
from python_modules.muonpipe import usefull_func
from matplotlib.patches import Circle
start_date_2020 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

In [ ]:
usefull_func.atmo_thick(4280)

In [ ]:
# #muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/*')
# muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
# print(sorted(muon_files))
# #df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[0:5]] # [0:5] for 19-22 years, [5:] for 22-23 years
# df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)] 
# df_all_data = pd.concat(df_files, ignore_index=True)
# df_good_data = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['ring_containment'] > 0.9) & (df_all_data['size_outside'] < 500) & (df_all_data['ring_completeness'] > 0.9)]# & (df_all_data['ring_completeness'] > 0.8)]
# #df_good_data = df_all_data[(df_all_data['ring_containment'] > 0.999) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
# del(df_all_data)
# df_good_data.to_csv('/Users/vdk/muons2024/fits_complete_rings_low_nsb.csv', index=False)
# df_good_data


In [ ]:
#df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['ring_containment'] > 0.9) & (df_all_data['size_outside'] < 500) & (df_all_data['ring_completeness'] > 0.9)]
df_good_data = pd.read_csv('/Users/vdk/muons2024/fits_complete_rings_low_nsb.csv', na_values=['NA', '?'])
df_good_data

In [ ]:
working_frame = copy.deepcopy(df_good_data)
df_good_data = df_good_data[:1000000]

In [ ]:
mean_opt_eff = np.mean(df_good_data['muon_efficiency'])
mean_ring_width = np.mean(df_good_data['ring_width'])
mean_ring_radius = np.mean(df_good_data['ring_radius'])
mean_ring_size = np.mean(df_good_data['ring_size'])
mean_impact_parameter = np.mean(df_good_data['impact_parameter'])
print(np.mean(df_good_data['muon_efficiency']))
print(np.mean(df_good_data['ring_width']))
print(mean_ring_radius)

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_width'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Width of the ring, [deg]')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=(max(x_pos)-min(x_pos))/(len(x_pos)-5), label="Number of events")
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Size of the ring, [p.e.]')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=(max(x_pos)-min(x_pos))/(len(x_pos)-3), label="Number of events", alpha = 1)
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
df_good_data

In [ ]:
x_value_str = 'ring_width'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Optical efficiency value')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.set_ylim(0.1,0.3)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=(max(x_pos)-min(x_pos))/(len(x_pos)-2), label="Number of events")
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()
bx.set_xlim(0,0.2)

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Width of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Optical efficiency value')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=0.01, label="Number of events")
#bx.set_ylim(0,0.1)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'ring_size'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Optical efficiency value')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=100, label="Number of events")
#bx.set_ylim(0,0.1)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Size of the ring [p.e.]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'impact_parameter'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Optical efficiency value')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=0.1, label="Number of events")
#bx.set_ylim(0,0.1)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Impact parameters [m]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2

# Add the circle to the Axes
ax.add_patch(circle)

# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_good_data['impact_x_array'][:1200000], df_good_data['impact_y_array'][:1200000], alpha = 1, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
df_sorted = df_good_data.sort_values(by='event_time')
for i in range(0,15):
    start = int(eval(f'{i}e6'))
    end = int(eval(f'{i+1}e6'))
    plt.figure(figsize=(12,12))
    alpha = 0.1
    plt.scatter(df_sorted['impact_x_array'][start:end], df_sorted['impact_y_array'][start:end], alpha = alpha, s = 1, marker = 'x')
    plt.xlabel("Impact parameter X [m]")
    plt.ylabel("Impact parameter Y [m]")
    plt.title(f"Reconstructed impact points of the muons for {str(start)[0]}e6-{str(end)[0]}e6 events on the LST mirror, transparency of dots = {alpha}")
    plt.show()
    #plt.savefig(f'/Users/vdk/muons2024/images/low_nsb_all_runs/test/ImpactPoints_{i}e6-{i+1}e6_Alpha{alpha}_SORTED.png', dpi=200, format='png', bbox_inches='tight')
    plt.close()

In [ ]:
# plt.plot(df_sorted['event_time'])
iteration = 0
for index, row in df_sorted.iterrows():
    try:
        if row['event_time'] > df_sorted.iloc[index+1]['event_time']:
            iteration += 1
            print(f"FAIL because {row['event_time']} is greater then {df_sorted.iloc[index+1]['event_time']}, and this happend for {iteration} times")
    except:
        print(f"Number {index} and timer {row['event_time']} caused an error")

In [ ]:
2+3

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_width'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

use_for_bins = list(binned_data['bin_mid'])
use_for_hist = list(binned_data[y_value_str])

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Ring Width', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str],df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_ring_width, color='g', linestyle='--', label = f'Mean width for this period = {round(mean_ring_width, 3)}')
plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
plt.ylim(0,0.2)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsWidth_19-23Y.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Ring size, [p.e.]')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=0.01, label="Number of events")
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Width of the ring, [deg]')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=0.01, label="Number of events")
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
usefull_func.hist_wo_outliers(df_good_data['ring_radius'],outlier_value=2)

### Simulations

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0684-reflection/tables/*')
#listdir2 = glob.glob('/Users/vdk/muons2024/cscs_simulations/tables_large_lst_muons/*')
#listdir3 = glob.glob('/Users/vdk/muons2024/cscs_simulations/muons_zenith0/*')
#istdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0684-reflection/tables/*')
# Muons analyzed using GlobalPeakWindowSum
#listdir=listdir+listdir2+listdir3
dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])

        
df = dat.to_pandas()
df_good_data = df[df['good_ring']]# & df['muon_efficiency'] < 1]
print(np.mean(df_good_data['muon_efficiency']))
df_good_data